# Stroke Prediction
   In this project we will be predicting that the person will be getting heart stoke or not with the help of features like age, gender, smoking status, bmi, etc...

So we will be doing the following steps as followed in life cycle
1. Exploring data
1. Feature Engineering
1. Creating a Deep Learning model using Keras
1. traing the model with best set of Hidden Layers and Neurons
1. evaluating the Model

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Getting data

In [ ]:
df = pd.read_csv("../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv")

### Exploring Data

In [ ]:
sns.pairplot(df, hue='stroke')

### Feature Engineering
   Removing the outliers and converting the classification features to continues features.

In [ ]:
df = df[df['gender']!='Other']
df['bmi'].fillna(df['bmi'].mean(),inplace=True)
df.drop('id',axis=1, inplace=True)
work_dict = {}
j=1
for i in df['work_type'].unique():
   work_dict[i]=j
   j+=1
df['work_type'] = df['work_type'].apply(lambda x: work_dict[x])
df = pd.get_dummies(df, columns=['gender','Residence_type','ever_married','smoking_status'],drop_first=True)

In [ ]:
print(df.head())
print(df.info())

### Model Creation

In [ ]:
import keras
from keras import Sequential
from keras.layers import Dense
from keras.losses import binary_crossentropy
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
smote = SMOTE()

In [ ]:
X = df.drop('stroke',axis=1)
y = df['stroke']
X, y = smote.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=22)

In [ ]:
model = Sequential()
model.add(Dense(5, input_dim=X.shape[1], activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, verbose=0, epochs = 40, batch_size=64, validation_split=0.33)

### Model Evaluation 

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model.evaluate(X_test,y_test)

### Model Accuracy

In [ ]:
pred=[]
for i in model.predict(X_test).reshape(-1): pred.append(round(i))
from sklearn.metrics import classification_report
print(classification_report(y_test, pred))

### Summary

   So finally we have created a Deep Learning Model for the stroke prediction, which will be predicting at an acuracy of 80%-85%.